# Setup AML

In this notebook, we'll setup our Azure Machine Learning workspace as well as another storage account.

---

### AML Installation and configuration
This notebook configures the notebooks in this tutorial to connect to an Azure Machine Learning (AML) Workspace.  You can use an existing workspace or create a new one.

In [1]:
import azureml.core
from azureml.core import Workspace
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os
from utilities import get_auth

In [2]:
print("AML SDK Version:", azureml.core.VERSION) # 1.0.2

AML SDK Version: 1.0.17


### Set up your Azure Machine Learning workspace

To create or access an Azure ML Workspace, you will need the following information:

* Your subscription id
* A resource group name
* A name for your workspace
* A region for your workspace

**Note**: As with other Azure services, there are limits on certain resources like cluster size associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

If you have a workspace created already, you need to get your subscription and workspace information. You can find the values for those by visiting your workspace in the [Azure portal](http://portal.azure.com). If you don't have a workspace, the create workspace command in the next section will create a resource group and a workspace using the names you provide.

 Replace the values in the following cell with your information.
 
 - you can navigate to the portal to find your subscription id or enter `az account list -o table` into the cli
 - you can use `az account list-locations -o table` to find the available locations (use the values in the field 'name')

In [5]:
subscription_id = "<YOUR-SUBSCRIPTION>"
resource_group = "<YOUR-RESOURCE-GROUP-NAME>"
workspace_name = "<YOUR-WORKSPACE-NAME>"
workspace_region = "<YOUR-REGION>"
storage_account_name = "<YOUR-STORAGE-ACCOUNT-NAME>"

Create and initialize a dotenv file for storing parameters used in multiple notebooks.

In [6]:
env_path = find_dotenv()
if env_path == "":
    Path(".env").touch()
    env_path = find_dotenv()

In [7]:
set_key(env_path, "SUBSCRIPTION_ID", subscription_id) 
set_key(env_path, "RESOURCE_GROUP", resource_group)
set_key(env_path, "WORKSPACE_NAME", workspace_name)
set_key(env_path, "REGION", workspace_region)

(True, 'REGION', 'eastus')

### Create the workspace
This cell will create an AML workspace for you in a subscription, provided you have the correct permissions.

This will fail when:
1. You do not have permission to create a workspace in the resource group
2. You do not have permission to create a resource group if it's non-existing.
2. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources. If this cell succeeds, you're done configuring AML!  

In [ ]:
# import the Workspace class
ws = Workspace.create(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group,
    location=workspace_region,
    create_resource_group=True,
    exist_ok=True,
    auth=get_auth()
)

Let's check the details of the workspace.

In [ ]:
ws.get_details()

Let's write the workspace configuration for the rest of the notebooks to connect to the workspace.

In [10]:
ws.write_config()

Wrote the config file config.json to: /home/mat/repos/Batch-Scoring-Deep-Learning-Models-With-AML/notebooks/aml_config/config.json


At this point, you can inspect your resource group to see your active Azure Machine Learning workspace. Inside your resource group, you'll find various resources that have been generated to back the Machine Learning workspace. These resources will have a generated resource name, such as `<workspace-name>keyvaultpuvogqzk`.

### Setup a Storage Account

One of the resources that will have been generated when created the workspace is a storage account. This storage account is used as the default datastore for Azure Machine Learning. However, to keep things seperate, we'll create another storage account for holding input and output data. This will also let us walk through how to connect our workspace to other storage locations.

In [11]:
storage_container_name = "aml"

Use the __az cli__ to create the account

In [12]:
!az storage account create \
    -n {storage_account_name} \
    -g {resource_group} \
    --query 'provisioningState'

"Succeeded" ..


Use the __az cli__ to grab the keys of the storage account that was just created. The `--quote '[0].value'` part of the command simply means to select the _value_ of the _zero-th indexed_ of the set of keys.

In [13]:
key = !az storage account keys list \
    --account-name {storage_account_name} \
    -g {resource_group} \
    --query '[0].value'

The stdout from the command above is stored in a string array of 1. Select the element in the array and ttrip opening and closing quotation marks.

In [14]:
storage_account_key = str(key[0][1:-1]) # this is used to strip opening and closing quotation marks

Use the __az cli__ to create the container in the storage account

In [15]:
!az storage container create \
    --account-name {storage_account_name} \
    --account-key {storage_account_key} \
    --name {storage_container_name}

{
  "created": true
}


Set storage account key to dotenv file

In [16]:
set_key(env_path, "STORAGE_ACCOUNT_NAME", storage_account_name)
set_key(env_path, "STORAGE_ACCOUNT_KEY", storage_account_key) # generated
set_key(env_path, "STORAGE_CONTAINER_NAME", storage_container_name)

(True, 'STORAGE_CONTAINER_NAME', 'aml')

---

You are now ready to move on to the [next notebook](03_develop_pipeline.ipynb).